In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 

In [2]:
df = pd.read_csv("../Output/topic_keyword_freq_NDC_sentences_CN.csv")
df['num_words'] = df['text_content'].apply(lambda x: len(x.split()))
df

,iso,country,ndc,date,html,text_type,text_content,h1_text,h2_text,h3_text,...,fisheries,climate change,industry,policy,biodiversity,ocean ecosystems,coastal impacts,blue economy,pollutants/waste,num_words
0,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,p,21 September 2015,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,...,0,0,0,0,0,0,0,0,0,3
1,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,p,The Islamic Republic of Afghanistan hereby com...,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,...,0,0,0,0,0,0,0,0,0,20
2,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,table,Base Year: 2005 Target Years: 2020 to 2030 Con...,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,...,0,0,0,1,0,0,0,0,0,57
3,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,table,Financial Needs: Total: USD 17.405 billionAdap...,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,...,0,0,0,0,0,0,0,0,0,13
4,AFG,Afghanistan,1,11/23/16,AFG-first_ndc-EN.html,ul,Adaptation: USD 10.785 billion Mitigation: USD...,Intended Nationally Determined Contribution,Submission to the United Nations Framework Con...,NaN,...,0,0,0,0,0,0,0,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161026,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,ol,This is likely due to the lower estimates of e...,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,...,0,0,0,0,0,0,0,0,0,83
161027,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,ol,This change is due to the fact that the previo...,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,...,0,0,0,0,0,0,0,0,0,21
161028,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,ol,Zimbabwe is now using updated data on emission...,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,...,0,0,0,0,0,0,0,0,0,72
161029,ZWE,Zimbabwe,2,9/24/21,ZWE-revised_first_ndc-EN.html,ol,Gender Analysis of NDCs.,GOVERNMENT OF ZIMBABWE,"7 Information for Clarity, Transparency and Un...",4.6.4 Waste,...,0,0,0,0,0,0,0,0,0,4


In [3]:
grouped = df.groupby(["iso","country", "ndc", "date"]).sum()
grouped

C:\Users\User\AppData\Local\Temp\ipykernel_18788\2547426715.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = df.groupby(["iso","country", "ndc", "date"]).sum()


global warming  ocean warming  acidification  \
iso country      ndc date                                                     
AFG Afghanistan  1   11/23/16               1              0              0   
AGO Angola       1   11/16/20               3              0              0   
                 2   5/31/21                8              0              7   
ALB Albania      1   9/21/16                1              0              0   
                 2   10/12/21               7              0              0   
...                                       ...            ...            ...   
ZAF South Africa 2   9/27/21               10              0              0   
ZMB Zambia       1   12/9/16                1              0              0   
                 2   7/30/21                2              0              0   
ZWE Zimbabwe     1   8/7/17                 1              0              0   
                 2   9/24/21                5              0              0   

                               sealevel rise  deoxygenation  ice melt  \
iso country      ndc date                                               
AFG Afghanistan  1   11/23/16              0              0         0   
AGO Angola       1   11/16/20              0              0         0   
                 2   5/31/21               0              0         0   
ALB Albania      1   9/21/16               0              0         0   
                 2   10/12/21              4              0         0   
...                                      ...            ...       ...   
ZAF South Africa 2   9/27/21               2              0         0   
ZMB Zambia       1   12/9/16               0              0         0   
                 2   7/30/21               0              0         0   
ZWE Zimbabwe     1   8/7/17                0              0         0   
                 2   9/24/21               0              0         0   

                               natural disasters  coast  coastal development  \
iso country      ndc date                                                      
AFG Afghanistan  1   11/23/16                  1      0                    0   
AGO Angola       1   11/16/20                  2     12                    0   
                 2   5/31/21                   1     71                    0   
ALB Albania      1   9/21/16                   0      0                    0   
                 2   10/12/21                  5    281                    0   
...                                          ...    ...                  ...   
ZAF South Africa 2   9/27/21                   0      9                    0   
ZMB Zambia       1   12/9/16                   0      0                    0   
                 2   7/30/21                   0      2                    0   
ZWE Zimbabwe     1   8/7/17                    0      0                    0   
                 2   9/24/21                   0      2                    0   

                               sealevel  ...  fisheries  climate change  \
iso country      ndc date                ...                              
AFG Afghanistan  1   11/23/16         0  ...          0               2   
AGO Angola       1   11/16/20         1  ...          0               5   
                 2   5/31/21          0  ...          6              23   
ALB Albania      1   9/21/16          0  ...          0               1   
                 2   10/12/21         6  ...         80              16   
...                                 ...  ...        ...             ...   
ZAF South Africa 2   9/27/21          2  ...          0              12   
ZMB Zambia       1   12/9/16          0  ...          2               1   
                 2   7/30/21          0  ...          3               2   
ZWE Zimbabwe     1   8/7/17           0  ...          0               1   
                 2   9/24/21          0  ...          0               5   

                               indus

In [4]:
keywords_df = pd.read_csv("../Data/keywords_for_marine_policy.csv")
keywords = list(set(keywords_df['Keyword']))
topics = list(set(keywords_df['Topic']))

In [5]:
for topic in topics:
    grouped[topic + "_freq_per_10000"] = (grouped[topic] / grouped["num_words"]) * 10000
for keyword in keywords:
    grouped[keyword + "_freq_per_10000"] = (grouped[keyword] / grouped["num_words"]) * 10000
grouped['keyword_freq_per_10000'] = (grouped[keywords].sum(axis=1) / grouped['num_words']) * 10000
grouped 

global warming  ocean warming  acidification  \
iso country      ndc date                                                     
AFG Afghanistan  1   11/23/16               1              0              0   
AGO Angola       1   11/16/20               3              0              0   
                 2   5/31/21                8              0              7   
ALB Albania      1   9/21/16                1              0              0   
                 2   10/12/21               7              0              0   
...                                       ...            ...            ...   
ZAF South Africa 2   9/27/21               10              0              0   
ZMB Zambia       1   12/9/16                1              0              0   
                 2   7/30/21                2              0              0   
ZWE Zimbabwe     1   8/7/17                 1              0              0   
                 2   9/24/21                5              0              0   

                               sealevel rise  deoxygenation  ice melt  \
iso country      ndc date                                               
AFG Afghanistan  1   11/23/16              0              0         0   
AGO Angola       1   11/16/20              0              0         0   
                 2   5/31/21               0              0         0   
ALB Albania      1   9/21/16               0              0         0   
                 2   10/12/21              4              0         0   
...                                      ...            ...       ...   
ZAF South Africa 2   9/27/21               2              0         0   
ZMB Zambia       1   12/9/16               0              0         0   
                 2   7/30/21               0              0         0   
ZWE Zimbabwe     1   8/7/17                0              0         0   
                 2   9/24/21               0              0         0   

                               natural disasters  coast  coastal development  \
iso country      ndc date                                                      
AFG Afghanistan  1   11/23/16                  1      0                    0   
AGO Angola       1   11/16/20                  2     12                    0   
                 2   5/31/21                   1     71                    0   
ALB Albania      1   9/21/16                   0      0                    0   
                 2   10/12/21                  5    281                    0   
...                                          ...    ...                  ...   
ZAF South Africa 2   9/27/21                   0      9                    0   
ZMB Zambia       1   12/9/16                   0      0                    0   
                 2   7/30/21                   0      2                    0   
ZWE Zimbabwe     1   8/7/17                    0      0                    0   
                 2   9/24/21                   0      2                    0   

                               sealevel  ...  hypoxia_freq_per_10000  \
iso country      ndc date                ...                           
AFG Afghanistan  1   11/23/16         0  ...                     0.0   
AGO Angola       1   11/16/20         1  ...                     0.0   
                 2   5/31/21          0  ...                     0.0   
ALB Albania      1   9/21/16          0  ...                     0.0   
                 2   10/12/21         6  ...                     0.0   
...                                 ...  ...                     ...   
ZAF South Africa 2   9/27/21          2  ...                     0.0   
ZMB Zambia       1   12/9/16          0  ...                     0.0   
                 2   7/30/21          0  ...                     0.0   
ZWE Zimbabwe     1   8/7/17           0  ...                     0.0   
                 2   9/24/21          0  ...                     0.0   

                               seabirds_freq_per_10000  \
iso country      

In [6]:
grouped = grouped.drop(columns=keywords)
grouped = grouped.drop(columns=topics)
grouped

num_words  climate change_freq_per_10000  \
iso country      ndc date                                                 
AFG Afghanistan  1   11/23/16       3564                       5.611672   
AGO Angola       1   11/16/20       7389                       6.766816   
                 2   5/31/21       27976                       8.221333   
ALB Albania      1   9/21/16        1132                       8.833922   
                 2   10/12/21      68451                       2.337438   
...                                  ...                            ...   
ZAF South Africa 2   9/27/21       19955                       6.013530   
ZMB Zambia       1   12/9/16        4664                       2.144082   
                 2   7/30/21        9781                       2.044781   
ZWE Zimbabwe     1   8/7/17         4227                       2.365744   
                 2   9/24/21       24967                       2.002643   

                               pollutants/waste_freq_per_10000  \
iso country      ndc date                                        
AFG Afghanistan  1   11/23/16                         8.417508   
AGO Angola       1   11/16/20                         2.706726   
                 2   5/31/21                          3.574492   
ALB Albania      1   9/21/16                          0.000000   
                 2   10/12/21                        10.810653   
...                                                        ...   
ZAF South Africa 2   9/27/21                          0.000000   
ZMB Zambia       1   12/9/16                         21.440823   
                 2   7/30/21                          8.179123   
ZWE Zimbabwe     1   8/7/17                           0.000000   
                 2   9/24/21                          2.803701   

                               coastal impacts_freq_per_10000  \
iso country      ndc date                                       
AFG Afghanistan  1   11/23/16                        0.000000   
AGO Angola       1   11/16/20                       28.420625   
                 2   5/31/21                        27.523592   
ALB Albania      1   9/21/16                         0.000000   
                 2   10/12/21                       59.750771   
...                                                       ...   
ZAF South Africa 2   9/27/21                         5.512403   
ZMB Zambia       1   12/9/16                         0.000000   
                 2   7/30/21                         2.044781   
ZWE Zimbabwe     1   8/7/17                          0.000000   
                 2   9/24/21                         5.607402   

                               policy_freq_per_10000  \
iso country      ndc date                              
AFG Afghanistan  1   11/23/16              61.728395   
AGO Angola       1   11/16/20              31.127351   
                 2   5/31/21               55.762082   
ALB Albania      1   9/21/16               26.501767   
                 2   10/12/21              55.368074   
...                                              ...   
ZAF South Africa 2   9/27/21               38.586820   
ZMB Zambia       1   12/9/16               60.034305   
                 2   7/30/21               32.716491   
ZWE Zimbabwe     1   8/7/17                42.583392   
                 2   9/24/21               32.042296   

                               ocean ecosystems_freq_per_10000  \
iso country      ndc date                                        
AFG Afghanistan  1   11/23/16                         8.417508   
AGO Angola       1   11/16/20                        17.593720   
                 2   5/31/21                         38.961967   
ALB Albania      1   9/21/16                          0.000000   
                 2   10/12/21                        51.861916   
...                                                        ...   
ZAF South Africa 2   9/27/21                          7.015786   
ZMB Zambia       1   12/9/1

In [7]:
grouped = grouped.reset_index()
grouped

,iso,country,ndc,date,num_words,climate change_freq_per_10000,pollutants/waste_freq_per_10000,coastal impacts_freq_per_10000,policy_freq_per_10000,ocean ecosystems_freq_per_10000,...,hypoxia_freq_per_10000,seabirds_freq_per_10000,blue carbon_freq_per_10000,ocean_freq_per_10000,microplastics_freq_per_10000,coast_freq_per_10000,oil spills_freq_per_10000,fish_freq_per_10000,pelagic_freq_per_10000,keyword_freq_per_10000
0,AFG,Afghanistan,1,11/23/16,3564,5.611672,8.417508,0.000000,61.728395,8.417508,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,101.010101
1,AGO,Angola,1,11/16/20,7389,6.766816,2.706726,28.420625,31.127351,17.593720,...,0.0,0.0,0.0,0.000000,0.0,16.240357,0.0,10.826905,0.000000,104.208959
2,AGO,Angola,2,5/31/21,27976,8.221333,3.574492,27.523592,55.762082,38.961967,...,0.0,0.0,0.0,3.217043,0.0,25.378896,0.0,22.161853,0.000000,180.154418
3,ALB,Albania,1,9/21/16,1132,8.833922,0.000000,0.000000,26.501767,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,35.335689
4,ALB,Albania,2,10/12/21,68451,2.337438,10.810653,59.750771,55.368074,51.861916,...,0.0,0.0,0.0,0.292180,0.0,41.051263,0.0,30.240610,0.876539,230.091598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,ZAF,South Africa,2,9/27/21,19955,6.013530,0.000000,5.512403,38.586820,7.015786,...,0.0,0.0,0.0,0.000000,0.0,4.510148,0.0,6.013530,0.000000,63.142070
298,ZMB,Zambia,1,12/9/16,4664,2.144082,21.440823,0.000000,60.034305,21.440823,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,8.576329,0.000000,150.085763
299,ZMB,Zambia,2,7/30/21,9781,2.044781,8.179123,2.044781,32.716491,3.067171,...,0.0,0.0,0.0,0.000000,0.0,2.044781,0.0,7.156732,0.000000,58.276250
300,ZWE,Zimbabwe,1,8/7/17,4227,2.365744,0.000000,0.000000,42.583392,28.388928,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,75.703809


In [8]:
grouped[grouped['iso'] == 'SSD']

,iso,country,ndc,date,num_words,climate change_freq_per_10000,pollutants/waste_freq_per_10000,coastal impacts_freq_per_10000,policy_freq_per_10000,ocean ecosystems_freq_per_10000,...,hypoxia_freq_per_10000,seabirds_freq_per_10000,blue carbon_freq_per_10000,ocean_freq_per_10000,microplastics_freq_per_10000,coast_freq_per_10000,oil spills_freq_per_10000,fish_freq_per_10000,pelagic_freq_per_10000,keyword_freq_per_10000
244,SSD,South Sudan,1,2/23/21,6259,0.000000,3.195399,9.586196,23.965490,4.793098,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.195399,0.0,52.724077
245,SSD,South Sudan,2,9/21/21,75209,1.861479,14.625909,35.235145,38.825141,24.731083,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.672938,0.0,146.126129


In [9]:
country_to_code = pd.read_csv("https://gist.githubusercontent.com/radcliff/f09c0f88344a7fcef373/raw/2753c482ad091c54b1822288ad2e4811c021d8ec/wikipedia-iso-country-codes.csv")
country_to_code

,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS
...,...,...,...,...,...
241,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF
242,Western Sahara,EH,ESH,732,ISO 3166-2:EH
243,Yemen,YE,YEM,887,ISO 3166-2:YE
244,Zambia,ZM,ZMB,894,ISO 3166-2:ZM


In [10]:
country_to_code.astype({"Numeric code": object})
def three_dig_num(num):
    num = str(num)
    if len(num) == 1:
        return "00" + num 
    elif len(num) == 2:
        return "0" + num
    else:
        return num 
country_to_code["Numeric code"] = country_to_code["Numeric code"].apply(three_dig_num)
country_to_code


,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,004,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,008,ISO 3166-2:AL
3,Algeria,DZ,DZA,012,ISO 3166-2:DZ
4,American Samoa,AS,ASM,016,ISO 3166-2:AS
...,...,...,...,...,...
241,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF
242,Western Sahara,EH,ESH,732,ISO 3166-2:EH
243,Yemen,YE,YEM,887,ISO 3166-2:YE
244,Zambia,ZM,ZMB,894,ISO 3166-2:ZM


In [11]:
cool_dict = {country_to_code.loc[i,"Alpha-3 code"]: country_to_code.loc[i, "Numeric code"] for i in country_to_code.index}
cool_dict["SSD"] = 728
cool_dict

{'AFG': '004',
 'ALA': '248',
 'ALB': '008',
 'DZA': '012',
 'ASM': '016',
 'AND': '020',
 'AGO': '024',
 'AIA': '660',
 'ATA': '010',
 'ATG': '028',
 'ARG': '032',
 'ARM': '051',
 'ABW': '533',
 'AUS': '036',
 'AUT': '040',
 'AZE': '031',
 'BHS': '044',
 'BHR': '048',
 'BGD': '050',
 'BRB': '052',
 'BLR': '112',
 'BEL': '056',
 'BLZ': '084',
 'BEN': '204',
 'BMU': '060',
 'BTN': '064',
 'BOL': '068',
 'BIH': '070',
 'BWA': '072',
 'BVT': '074',
 'BRA': '076',
 'IOT': '086',
 'BRN': '096',
 'BGR': '100',
 'BFA': '854',
 'BDI': '108',
 'KHM': '116',
 'CMR': '120',
 'CAN': '124',
 'CPV': '132',
 'CYM': '136',
 'CAF': '140',
 'TCD': '148',
 'CHL': '152',
 'CHN': '156',
 'CXR': '162',
 'CCK': '166',
 'COL': '170',
 'COM': '174',
 'COG': '178',
 'COD': '180',
 'COK': '184',
 'CRI': '188',
 'CIV': '384',
 'HRV': '191',
 'CUB': '192',
 'CYP': '196',
 'CZE': '203',
 'DNK': '208',
 'DJI': '262',
 'DMA': '212',
 'DOM': '214',
 'ECU': '218',
 'EGY': '818',
 'SLV': '222',
 'GNQ': '226',
 'ERI': '2

In [12]:
grouped['numeric_code'] = grouped['iso'].apply(lambda x: cool_dict[x])
grouped

,iso,country,ndc,date,num_words,climate change_freq_per_10000,pollutants/waste_freq_per_10000,coastal impacts_freq_per_10000,policy_freq_per_10000,ocean ecosystems_freq_per_10000,...,seabirds_freq_per_10000,blue carbon_freq_per_10000,ocean_freq_per_10000,microplastics_freq_per_10000,coast_freq_per_10000,oil spills_freq_per_10000,fish_freq_per_10000,pelagic_freq_per_10000,keyword_freq_per_10000,numeric_code
0,AFG,Afghanistan,1,11/23/16,3564,5.611672,8.417508,0.000000,61.728395,8.417508,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,101.010101,004
1,AGO,Angola,1,11/16/20,7389,6.766816,2.706726,28.420625,31.127351,17.593720,...,0.0,0.0,0.000000,0.0,16.240357,0.0,10.826905,0.000000,104.208959,024
2,AGO,Angola,2,5/31/21,27976,8.221333,3.574492,27.523592,55.762082,38.961967,...,0.0,0.0,3.217043,0.0,25.378896,0.0,22.161853,0.000000,180.154418,024
3,ALB,Albania,1,9/21/16,1132,8.833922,0.000000,0.000000,26.501767,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,35.335689,008
4,ALB,Albania,2,10/12/21,68451,2.337438,10.810653,59.750771,55.368074,51.861916,...,0.0,0.0,0.292180,0.0,41.051263,0.0,30.240610,0.876539,230.091598,008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,ZAF,South Africa,2,9/27/21,19955,6.013530,0.000000,5.512403,38.586820,7.015786,...,0.0,0.0,0.000000,0.0,4.510148,0.0,6.013530,0.000000,63.142070,710
298,ZMB,Zambia,1,12/9/16,4664,2.144082,21.440823,0.000000,60.034305,21.440823,...,0.0,0.0,0.000000,0.0,0.000000,0.0,8.576329,0.000000,150.085763,894
299,ZMB,Zambia,2,7/30/21,9781,2.044781,8.179123,2.044781,32.716491,3.067171,...,0.0,0.0,0.000000,0.0,2.044781,0.0,7.156732,0.000000,58.276250,894
300,ZWE,Zimbabwe,1,8/7/17,4227,2.365744,0.000000,0.000000,42.583392,28.388928,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,75.703809,716


In [13]:
grouped['date_ndc'] = pd.to_datetime(grouped['date'])
most_recent_ndcs = grouped.loc[grouped.groupby('country')['date_ndc'].idxmax()]
most_recent_ndcs = most_recent_ndcs.drop(columns=["date_ndc"])
most_recent_ndcs

,iso,country,ndc,date,num_words,climate change_freq_per_10000,pollutants/waste_freq_per_10000,coastal impacts_freq_per_10000,policy_freq_per_10000,ocean ecosystems_freq_per_10000,...,seabirds_freq_per_10000,blue carbon_freq_per_10000,ocean_freq_per_10000,microplastics_freq_per_10000,coast_freq_per_10000,oil spills_freq_per_10000,fish_freq_per_10000,pelagic_freq_per_10000,keyword_freq_per_10000,numeric_code
0,AFG,Afghanistan,1,11/23/16,3564,5.611672,8.417508,0.000000,61.728395,8.417508,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,101.010101,004
4,ALB,Albania,2,10/12/21,68451,2.337438,10.810653,59.750771,55.368074,51.861916,...,0.0,0.0,0.292180,0.0,41.051263,0.0,30.240610,0.876539,230.091598,008
82,DZA,Algeria,1,10/20/16,3222,9.310987,0.000000,9.310987,34.140286,6.207325,...,0.0,0.0,0.000000,0.0,9.310987,0.0,0.000000,0.000000,65.176909,012
6,AND,Andorra,2,5/20/20,13452,3.716919,1.486768,1.486768,50.550104,5.947071,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,65.417782,020
2,AGO,Angola,2,5/31/21,27976,8.221333,3.574492,27.523592,55.762082,38.961967,...,0.0,0.0,3.217043,0.0,25.378896,0.0,22.161853,0.000000,180.154418,024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,VAT,Vatican,1,5/31/23,2447,12.259910,0.000000,0.000000,69.472824,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,89.906007,336
289,VEN,Venezuela,2,11/9/21,104202,1.631447,6.525786,18.329783,59.691753,24.279764,...,0.0,0.0,1.055642,0.0,14.299150,0.0,0.095967,0.000000,126.101226,862
291,VNM,Viet Nam,2,9/11/20,17392,26.448942,1.149954,25.298988,48.873045,44.273229,...,0.0,0.0,0.000000,0.0,13.799448,0.0,5.174793,0.000000,161.568537,704
299,ZMB,Zambia,2,7/30/21,9781,2.044781,8.179123,2.044781,32.716491,3.067171,...,0.0,0.0,0.000000,0.0,2.044781,0.0,7.156732,0.000000,58.276250,894


In [14]:
def in_EU(string):
    return string in ["AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"]

In [15]:
iso_to_country = {
    "BEL": "Belgium",
    "BGR": "Bulgaria",
    "HRV": "Croatia",
    "CYP": "Cyprus",
    "CZE": "Czech Republic",
    "DNK": "Denmark",
    "EST": "Estonia", 
    "FIN": "Finland",
    "FRA": "France",
    "DEU": "Germany",
    "GRC": "Greece",
    "HUN": "Hungary",
    "IRL": "Ireland",
    "ITA": "Italy",
    "LVA": "Latvia", 
    "LTU": "Lithuania", 
    "LUX": "Luxembourg", 
    "MLT": "Malta", 
    "NLD": "Netherlands", 
    "POL": "Poland",
    "PRT": "Portugal",
    "ROU": "Romania", 
    "SVK": "Slovakia",
    "SVN": "Slovenia",
    "ESP": "Spain",
    "SWE": "Sweden"
}

In [16]:
most_recent_ndcs = most_recent_ndcs.reset_index(drop=True)
most_recent_ndcs

,iso,country,ndc,date,num_words,climate change_freq_per_10000,pollutants/waste_freq_per_10000,coastal impacts_freq_per_10000,policy_freq_per_10000,ocean ecosystems_freq_per_10000,...,seabirds_freq_per_10000,blue carbon_freq_per_10000,ocean_freq_per_10000,microplastics_freq_per_10000,coast_freq_per_10000,oil spills_freq_per_10000,fish_freq_per_10000,pelagic_freq_per_10000,keyword_freq_per_10000,numeric_code
0,AFG,Afghanistan,1,11/23/16,3564,5.611672,8.417508,0.000000,61.728395,8.417508,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,101.010101,004
1,ALB,Albania,2,10/12/21,68451,2.337438,10.810653,59.750771,55.368074,51.861916,...,0.0,0.0,0.292180,0.0,41.051263,0.0,30.240610,0.876539,230.091598,008
2,DZA,Algeria,1,10/20/16,3222,9.310987,0.000000,9.310987,34.140286,6.207325,...,0.0,0.0,0.000000,0.0,9.310987,0.0,0.000000,0.000000,65.176909,012
3,AND,Andorra,2,5/20/20,13452,3.716919,1.486768,1.486768,50.550104,5.947071,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,65.417782,020
4,AGO,Angola,2,5/31/21,27976,8.221333,3.574492,27.523592,55.762082,38.961967,...,0.0,0.0,3.217043,0.0,25.378896,0.0,22.161853,0.000000,180.154418,024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,VAT,Vatican,1,5/31/23,2447,12.259910,0.000000,0.000000,69.472824,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,89.906007,336
163,VEN,Venezuela,2,11/9/21,104202,1.631447,6.525786,18.329783,59.691753,24.279764,...,0.0,0.0,1.055642,0.0,14.299150,0.0,0.095967,0.000000,126.101226,862
164,VNM,Viet Nam,2,9/11/20,17392,26.448942,1.149954,25.298988,48.873045,44.273229,...,0.0,0.0,0.000000,0.0,13.799448,0.0,5.174793,0.000000,161.568537,704
165,ZMB,Zambia,2,7/30/21,9781,2.044781,8.179123,2.044781,32.716491,3.067171,...,0.0,0.0,0.000000,0.0,2.044781,0.0,7.156732,0.000000,58.276250,894


In [17]:
most_recent_ndcs[most_recent_ndcs['iso'].apply(in_EU)]

,iso,country,ndc,date,num_words,climate change_freq_per_10000,pollutants/waste_freq_per_10000,coastal impacts_freq_per_10000,policy_freq_per_10000,ocean ecosystems_freq_per_10000,...,seabirds_freq_per_10000,blue carbon_freq_per_10000,ocean_freq_per_10000,microplastics_freq_per_10000,coast_freq_per_10000,oil spills_freq_per_10000,fish_freq_per_10000,pelagic_freq_per_10000,keyword_freq_per_10000,numeric_code
51,AUT,European Union,2,12/18/20,11909,1.679402,0.0,5.038206,65.496683,3.358804,...,0.0,0.0,0.0,0.0,1.679402,0.0,1.679402,0.0,89.848014,040


In [18]:
for iso_code in iso_to_country:
    country_name = iso_to_country[iso_code]
    old_row = most_recent_ndcs.loc[51]
    old_row.loc['iso'] = iso_code 
    old_row.loc['country'] = country_name 
    old_row.loc['numeric_code'] = cool_dict[iso_code]
    most_recent_ndcs.loc[len(most_recent_ndcs)] = old_row
most_recent_ndcs

,iso,country,ndc,date,num_words,climate change_freq_per_10000,pollutants/waste_freq_per_10000,coastal impacts_freq_per_10000,policy_freq_per_10000,ocean ecosystems_freq_per_10000,...,seabirds_freq_per_10000,blue carbon_freq_per_10000,ocean_freq_per_10000,microplastics_freq_per_10000,coast_freq_per_10000,oil spills_freq_per_10000,fish_freq_per_10000,pelagic_freq_per_10000,keyword_freq_per_10000,numeric_code
0,AFG,Afghanistan,1,11/23/16,3564,5.611672,8.417508,0.000000,61.728395,8.417508,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,101.010101,004
1,ALB,Albania,2,10/12/21,68451,2.337438,10.810653,59.750771,55.368074,51.861916,...,0.0,0.0,0.292180,0.0,41.051263,0.0,30.240610,0.876539,230.091598,008
2,DZA,Algeria,1,10/20/16,3222,9.310987,0.000000,9.310987,34.140286,6.207325,...,0.0,0.0,0.000000,0.0,9.310987,0.0,0.000000,0.000000,65.176909,012
3,AND,Andorra,2,5/20/20,13452,3.716919,1.486768,1.486768,50.550104,5.947071,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,65.417782,020
4,AGO,Angola,2,5/31/21,27976,8.221333,3.574492,27.523592,55.762082,38.961967,...,0.0,0.0,3.217043,0.0,25.378896,0.0,22.161853,0.000000,180.154418,024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,ROU,Romania,2,12/18/20,11909,1.679402,0.000000,5.038206,65.496683,3.358804,...,0.0,0.0,0.000000,0.0,1.679402,0.0,1.679402,0.000000,89.848014,642
189,SVK,Slovakia,2,12/18/20,11909,1.679402,0.000000,5.038206,65.496683,3.358804,...,0.0,0.0,0.000000,0.0,1.679402,0.0,1.679402,0.000000,89.848014,703
190,SVN,Slovenia,2,12/18/20,11909,1.679402,0.000000,5.038206,65.496683,3.358804,...,0.0,0.0,0.000000,0.0,1.679402,0.0,1.679402,0.000000,89.848014,705
191,ESP,Spain,2,12/18/20,11909,1.679402,0.000000,5.038206,65.496683,3.358804,...,0.0,0.0,0.000000,0.0,1.679402,0.0,1.679402,0.000000,89.848014,724


In [19]:
most_recent_ndcs = most_recent_ndcs.sort_values(by="country").reset_index(drop=True)
most_recent_ndcs

,iso,country,ndc,date,num_words,climate change_freq_per_10000,pollutants/waste_freq_per_10000,coastal impacts_freq_per_10000,policy_freq_per_10000,ocean ecosystems_freq_per_10000,...,seabirds_freq_per_10000,blue carbon_freq_per_10000,ocean_freq_per_10000,microplastics_freq_per_10000,coast_freq_per_10000,oil spills_freq_per_10000,fish_freq_per_10000,pelagic_freq_per_10000,keyword_freq_per_10000,numeric_code
0,AFG,Afghanistan,1,11/23/16,3564,5.611672,8.417508,0.000000,61.728395,8.417508,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,101.010101,004
1,ALB,Albania,2,10/12/21,68451,2.337438,10.810653,59.750771,55.368074,51.861916,...,0.0,0.0,0.292180,0.0,41.051263,0.0,30.240610,0.876539,230.091598,008
2,DZA,Algeria,1,10/20/16,3222,9.310987,0.000000,9.310987,34.140286,6.207325,...,0.0,0.0,0.000000,0.0,9.310987,0.0,0.000000,0.000000,65.176909,012
3,AND,Andorra,2,5/20/20,13452,3.716919,1.486768,1.486768,50.550104,5.947071,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,65.417782,020
4,AGO,Angola,2,5/31/21,27976,8.221333,3.574492,27.523592,55.762082,38.961967,...,0.0,0.0,3.217043,0.0,25.378896,0.0,22.161853,0.000000,180.154418,024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,VAT,Vatican,1,5/31/23,2447,12.259910,0.000000,0.000000,69.472824,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,89.906007,336
189,VEN,Venezuela,2,11/9/21,104202,1.631447,6.525786,18.329783,59.691753,24.279764,...,0.0,0.0,1.055642,0.0,14.299150,0.0,0.095967,0.000000,126.101226,862
190,VNM,Viet Nam,2,9/11/20,17392,26.448942,1.149954,25.298988,48.873045,44.273229,...,0.0,0.0,0.000000,0.0,13.799448,0.0,5.174793,0.000000,161.568537,704
191,ZMB,Zambia,2,7/30/21,9781,2.044781,8.179123,2.044781,32.716491,3.067171,...,0.0,0.0,0.000000,0.0,2.044781,0.0,7.156732,0.000000,58.276250,894


In [20]:
aut_index = most_recent_ndcs[most_recent_ndcs['iso'] == "AUT"].index[0]
most_recent_ndcs.loc[aut_index, 'country'] = "Austria"
most_recent_ndcs

,iso,country,ndc,date,num_words,climate change_freq_per_10000,pollutants/waste_freq_per_10000,coastal impacts_freq_per_10000,policy_freq_per_10000,ocean ecosystems_freq_per_10000,...,seabirds_freq_per_10000,blue carbon_freq_per_10000,ocean_freq_per_10000,microplastics_freq_per_10000,coast_freq_per_10000,oil spills_freq_per_10000,fish_freq_per_10000,pelagic_freq_per_10000,keyword_freq_per_10000,numeric_code
0,AFG,Afghanistan,1,11/23/16,3564,5.611672,8.417508,0.000000,61.728395,8.417508,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,101.010101,004
1,ALB,Albania,2,10/12/21,68451,2.337438,10.810653,59.750771,55.368074,51.861916,...,0.0,0.0,0.292180,0.0,41.051263,0.0,30.240610,0.876539,230.091598,008
2,DZA,Algeria,1,10/20/16,3222,9.310987,0.000000,9.310987,34.140286,6.207325,...,0.0,0.0,0.000000,0.0,9.310987,0.0,0.000000,0.000000,65.176909,012
3,AND,Andorra,2,5/20/20,13452,3.716919,1.486768,1.486768,50.550104,5.947071,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,65.417782,020
4,AGO,Angola,2,5/31/21,27976,8.221333,3.574492,27.523592,55.762082,38.961967,...,0.0,0.0,3.217043,0.0,25.378896,0.0,22.161853,0.000000,180.154418,024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,VAT,Vatican,1,5/31/23,2447,12.259910,0.000000,0.000000,69.472824,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,89.906007,336
189,VEN,Venezuela,2,11/9/21,104202,1.631447,6.525786,18.329783,59.691753,24.279764,...,0.0,0.0,1.055642,0.0,14.299150,0.0,0.095967,0.000000,126.101226,862
190,VNM,Viet Nam,2,9/11/20,17392,26.448942,1.149954,25.298988,48.873045,44.273229,...,0.0,0.0,0.000000,0.0,13.799448,0.0,5.174793,0.000000,161.568537,704
191,ZMB,Zambia,2,7/30/21,9781,2.044781,8.179123,2.044781,32.716491,3.067171,...,0.0,0.0,0.000000,0.0,2.044781,0.0,7.156732,0.000000,58.276250,894


In [21]:
most_recent_ndcs[most_recent_ndcs["country"] == "Austria"]

,iso,country,ndc,date,num_words,climate change_freq_per_10000,pollutants/waste_freq_per_10000,coastal impacts_freq_per_10000,policy_freq_per_10000,ocean ecosystems_freq_per_10000,...,seabirds_freq_per_10000,blue carbon_freq_per_10000,ocean_freq_per_10000,microplastics_freq_per_10000,coast_freq_per_10000,oil spills_freq_per_10000,fish_freq_per_10000,pelagic_freq_per_10000,keyword_freq_per_10000,numeric_code
58,AUT,Austria,2,12/18/20,11909,1.679402,0.0,5.038206,65.496683,3.358804,...,0.0,0.0,0.0,0.0,1.679402,0.0,1.679402,0.0,89.848014,040


In [22]:
most_recent_ndcs.to_csv("../Output/topic_keyword_freq_by_countryndc_CN.csv", index=False)